In [1]:
import random
import numpy as np

from math import log
log2 = lambda x: log(x)/log(2)

# Tensors

In [2]:
# for arbitrary precision ints

DTYPE = 'object'
Q = 503928829565480993

# we need room for summing MAX_SUM values of MAX_DEGREE before during modulus reduction
MAX_DEGREE = 2
MAX_SUM = 2**10
assert MAX_DEGREE * log2(Q) + log2(MAX_SUM) < 128

BASE = 2
PRECISION_INTEGRAL   = 10
PRECISION_FRACTIONAL = 16
# TODO Gap as needed for local truncating

# we need room for double precision before truncating
assert PRECISION_INTEGRAL + 2 * PRECISION_FRACTIONAL < log(Q)/log(BASE)

In [3]:
# for arbitrary precision ints

DTYPE = 'object'
Q = 2657003489534545107915232808830590043

# we need room for summing MAX_SUM values of MAX_DEGREE before during modulus reduction
MAX_DEGREE = 2
MAX_SUM = 2**12
assert MAX_DEGREE * log2(Q) + log2(MAX_SUM) < 256

BASE = 2
PRECISION_INTEGRAL   = 16
PRECISION_FRACTIONAL = 32
# TODO Gap as needed for local truncating

# we need room for double precision before truncating
assert PRECISION_INTEGRAL + 2 * PRECISION_FRACTIONAL < log(Q)/log(BASE)

In [4]:
def encode(rationals):
    return (rationals * BASE**PRECISION_FRACTIONAL).astype('int').astype(DTYPE) % Q

def decode(elements):
    map_negative_range = np.vectorize(lambda element: element if element <= Q/2 else element - Q)
    return map_negative_range(elements) / BASE**PRECISION_FRACTIONAL

In [13]:
class PublicTensor:
    
    def __init__(self, values):
        self.values = values
    
    @property
    def shape(self):
        return self.values.shape
    
    def __repr__(self):
        return "PublicTensor(%s)" % decode(self.values)
    
    def load(values, apply_encoding=True):
        if apply_encoding: values = encode(values)
        return PublicTensor(values)
    
    def unwrap(self):
        return self.values
    
    def truncate(self, amount=PRECISION_FRACTIONAL):
        truncate = np.vectorize(lambda x: x // BASE**amount if x <= Q/2 else Q - ((Q - x) // BASE**amount))
        return PublicTensor(truncate(self.values))
    
    def add(x, y):
        if isinstance(y, PublicTensor):
            return PublicTensor((x.values + y.values) % Q)
        elif isinstance(y, PrivateTensor):
            shares0 = (x.values + y.shares0) % Q
            shares1 =             y.shares1
            return PrivateTensor(shares0, shares1)
        else:
            print(type(y))
            raise Error
        
    def __add__(x, y):
        return x.add(y)
        
    def sub(x, y):
        if isinstance(y, PublicTensor):
            return PublicTensor((x.values - y.values) % Q)
        elif isinstance(y, PrivateTensor):
            # TODO might be more efficient way
            shares0 = ((Q-1) * y.shares0 + x.values) % Q
            shares1 = ((Q-1) * y.shares1) % Q
            return PrivateTensor(shares0, shares1)
        elif isinstance(y, PublicTensor):
            shares0 = (x.shares0 - y.values) % Q
            shares1 =  x.shares1
            return PrivateTensor(shares0, shares1)
        else:
            print(type(y))
            raise Error
        
    def __sub__(x, y):
        return x.sub(y)
        
    def mul(x, y, truncate=True):
        if isinstance(y, PublicTensor):
            values = (x.values * y.values) % Q
            z = PublicTensor(values)
            if truncate: z = z.truncate()
            return z
        elif isinstance(y, PrivateTensor):
            shares0 = (x.values * y.shares0) % Q
            shares1 = (x.values * y.shares1) % Q
            z = PrivateTensor(shares0, shares1)
            if truncate: z = z.truncate()
            return z
        else:
            print(type(y))
            raise Error
    
    def __mul__(x, y):
        return x.mul(y)
    
    def div(x, y):
        if isinstance(y, PublicTensor):
            return PublicTensor(encode(decode(x.values) / decode(y.values)))
        elif isinstance(y, int) or isinstance(y, float):
            return PublicTensor(encode(decode(x.values) / y))
        elif isinstance(y, np.ndarray):
            return PublicTensor(encode(decode(x.values) / y))
        else:
            print(type(y))
            raise Error
        
    def __div__(x, y):
        return x.div(y)
    
    def dot(x, y, truncate=True):
        if isinstance(y, PublicTensor):
            z = PublicTensor(x.values.dot(y.values) % Q)
            if truncate: z = z.truncate()
            return z
        elif isinstance(y, PrivateTensor):
            shares0 = x.values.dot(y.shares0) % Q
            shares1 = x.values.dot(y.shares1) % Q
            z = PrivateTensor(shares0, shares1)
            if truncate: z = z.truncate()
            return z
        else:
            print(type(y))
            raise Error
        
    def transpose(x):
        return PublicTensor(x.values.T)
    
    def sum0(x):
        return PublicTensor(x.values.sum(axis=0, keepdims=True))
    
    def sum1(x):
        return PublicTensor(x.values.sum(axis=1, keepdims=True))
        
    def exp(x):
        return PublicTensor(encode(np.exp(decode(x.values))))

In [6]:
# x = PublicTensor.load(np.array([1,2,3,4]).reshape(4,1) - 2); print(x)
# y = PublicTensor.load(np.array([5,6,7,8]).reshape(4,1)); print(y)
# z = x + y; print(z)
# z = (x * y); print(z)
# z = x.dot(y.transpose()); print(z)
# z = z.sum0(); print(z)

In [7]:
def generate_mul_triple(shape):
    a = np.array([ random.randrange(Q) for _ in range(np.prod(shape)) ]).astype(DTYPE).reshape(shape)
    b = np.array([ random.randrange(Q) for _ in range(np.prod(shape)) ]).astype(DTYPE).reshape(shape)
    ab = (a * b) % Q
    return PrivateTensor.load(a, False), \
           PrivateTensor.load(b, False), \
           PrivateTensor.load(ab, False)

In [8]:
def generate_dot_triple(m, n, o):
    a = np.array([ random.randrange(Q) for _ in range(m*n) ]).astype(DTYPE).reshape((m,n))
    b = np.array([ random.randrange(Q) for _ in range(n*o) ]).astype(DTYPE).reshape((n,o))
    ab = np.dot(a, b)
    return PrivateTensor.load(a, False), \
           PrivateTensor.load(b, False), \
           PrivateTensor.load(ab, False)

In [9]:
class PrivateTensor:
    
    def __init__(self, shares0, shares1):
        self.shares0 = shares0
        self.shares1 = shares1

    def load(values, apply_encoding=True):
        if apply_encoding: values = encode(values)
        shares0 = np.array([ random.randrange(Q) for _ in range(values.size) ]).astype(DTYPE).reshape(values.shape)
        shares1 = (values - shares0) % Q
        return PrivateTensor(shares0, shares1)
    
    def unwrap(self):
        return (self.shares0 + self.shares1) % Q
    
    def reveal(self):
        values = (self.shares0 + self.shares1) % Q
        return PublicTensor(values)
    
    def truncate(self, amount=PRECISION_FRACTIONAL):
        shares0 = np.floor_divide(self.shares0, BASE**amount)
        shares1 = Q - (np.floor_divide(Q - self.shares1, BASE**amount))
        return PrivateTensor(shares0, shares1)
    
    def __repr__(self):
        return "PrivateTensor(%s)" % decode(self.reveal().values)
    
    @property
    def shape(self):
        return self.shares0.shape
    
    def add(x, y):
        if isinstance(y, PrivateTensor):
            shares0 = (x.shares0 + y.shares0) % Q
            shares1 = (x.shares1 + y.shares1) % Q
            return PrivateTensor(shares0, shares1)
        elif isinstance(y, PublicTensor):
            shares0 = (x.shares0 + y.values) % Q
            shares1 =  x.shares1
            return PrivateTensor(shares0, shares1)
        elif isinstance(y, np.ndarray):
            return x.add(PublicTensor.load(y))
        elif isinstance(y, int) or isinstance(y, float):
            return x.add(PublicTensor.load(np.array([y])))
        else:
            print(type(y))
            raise Error
        
    def __add__(x, y):
        return x.add(y)
    
    def sub(x, y):
        if isinstance(y, PrivateTensor):
            shares0 = (x.shares0 - y.shares0) % Q
            shares1 = (x.shares1 - y.shares1) % Q
            return PrivateTensor(shares0, shares1)
        elif isinstance(y, PublicTensor):
            shares0 = (x.shares0 - y.values) % Q
            shares1 =  x.shares1
            return PrivateTensor(shares0, shares1)
        elif isinstance(y, int):
            y = PublicTensor.load(np.array([y]))
            return x.sub(y)
        elif isinstance(y, float):
            y = PublicTensor.load(np.array([y]))
            return x.sub(y)
        else:
            print(type(y))
            raise Error
        
    def __sub__(x, y):
        return x.sub(y)
    
    def mul(x, y, truncate=True, precomputed=None):
        if isinstance(y, PrivateTensor):
            assert x.shares0.shape == x.shares1.shape == y.shares0.shape == y.shares1.shape
            if precomputed is None: precomputed = generate_mul_triple(x.shares0.shape)
            a, b, ab = precomputed
            assert x.shape == a.shape
            assert y.shape == b.shape
            alpha = (x - a).reveal()
            beta  = (y - b).reveal()
            z = alpha.mul(beta, truncate=False) + \
                alpha.mul(b, truncate=False) + \
                a.mul(beta, truncate=False) + \
                ab
            if truncate: z = z.truncate()
            return z
        elif isinstance(y, PublicTensor):
            shares0 = (x.shares0 * y.values) % Q
            shares1 = (x.shares1 * y.values) % Q
            z = PrivateTensor(shares0, shares1)
            if truncate: z = z.truncate()
            return z
        elif isinstance(y, np.ndarray):
            return x.mul(PublicTensor.load(y), truncate, precomputed)
        elif isinstance(y, int) or isinstance(y, float):
            return x.mul(PublicTensor.load(np.array([y])), truncate, precomputed)
        else:
            print(type(y))
            raise Error
        
    def __mul__(x, y):
        return x.mul(y)
        
    def dot(x, y, truncate=True, precomputed=None):
        if isinstance(y, PrivateTensor):
            assert x.shares0.shape == x.shares1.shape 
            assert y.shares0.shape == y.shares1.shape
            m = x.shares0.shape[0]
            n = x.shares0.shape[1]
            o = y.shares0.shape[1]
            assert n == y.shares0.shape[0]
            if precomputed is None: precomputed = generate_dot_triple(m, n, o)
            a, b, ab = precomputed
            alpha = (x - a).reveal()
            beta  = (y - b).reveal()
            z = alpha.dot(beta, truncate=False) + \
                alpha.dot(b, truncate=False) + \
                a.dot(beta, truncate=False) + \
                ab
            if truncate: z = z.truncate()
            return z
        elif isinstance(y, PublicTensor):
            shares0 = x.shares0.dot(y.values) % Q
            shares1 = x.shares1.dot(y.values) % Q
            z = PrivateTensor(shares0, shares1)
            if truncate: z = z.truncate()
            return z
        elif isinstance(y, np.ndarray):
            return x.dot(PublicTensor.load(y))
        else:
            print(type(y))
            raise Error
        
    def neg(self):
        return self.mul(PublicTensor(np.array([Q - 1])), truncate=False)
        
    def transpose(self):
        return PrivateTensor(self.shares0.T, self.shares1.T)
    
    def sum0(self):
        shares0 = self.shares0.sum(axis=0, keepdims=True) % Q
        shares1 = self.shares1.sum(axis=0, keepdims=True) % Q
        return PrivateTensor(shares0, shares1)

In [10]:
# x = PrivateTensor.load(np.array([1,2,3,4]).reshape(4,1) - 2)
# y = PrivateTensor.load(np.array([5,6,7,8]).reshape(4,1))
# z = x + y; print(z)
# z = (x * y); print(z)
# z = x.dot(y.transpose()); print(z)
# z = z.sum0(); print(z)

In [14]:
class NumpyTensor:
    
    def __init__(self, values):
        self.values = values
        
    @property
    def shape(self):
        return self.values.shape

    def load(values):
        return NumpyTensor(values)
    
    def unwrap(self):
        return self.values

    def reveal(self):
        return self
    
    def __repr__(self):
        return "NumpyTensor(%s)" % self.values
    
    def add(x, y):
        if isinstance(y, NumpyTensor):
            return NumpyTensor(x.values + y.values)
        elif isinstance(y, int):
            return NumpyTensor(x.values + y)
        elif isinstance(y, float):
            return NumpyTensor(x.values + y)
        elif isinstance(y, np.ndarray):
            return NumpyTensor(x.values + y)
        else:
            print(type(y))
            raise Error
        
    def __add__(x, y):
        return x.add(y)
    
    def sub(x, y):
        if isinstance(y, NumpyTensor):
            return NumpyTensor(x.values - y.values)
        elif isinstance(y, int):
            return NumpyTensor(x.values - y)
        elif isinstance(y, float):
            return NumpyTensor(x.values - y)
        elif isinstance(y, np.ndarray):
            return NumpyTensor(x.values - y)
        else:
            print(type(y))
            raise Error
        
    def __sub__(x, y):
        return x.sub(y)
    
    def mul(x, y):
        if isinstance(y, NumpyTensor):
            return NumpyTensor(x.values * y.values)
        elif isinstance(y, int):
            return NumpyTensor(x.values * y)
        elif isinstance(y, float):
            return NumpyTensor(x.values * y)
        elif isinstance(y, np.ndarray):
            return NumpyTensor(x.values * y)
        else:
            print(type(y))
            raise Error
        
    def __mul__(x, y):
        return x.mul(y)
    
    def div(x, y):
        if isinstance(y, NumpyTensor):
            return NumpyTensor(x.values / y.values)
        elif isinstance(y, int):
            return NumpyTensor(x.values / y)
        elif isinstance(y, float):
            return NumpyTensor(x.values / y)
        elif isinstance(y, np.ndarray):
            return NumpyTensor(x.values / y)
        else:
            print(type(y))
            raise Error
        
    def __div__(x, y):
        return x.div(y)
        
    def dot(x, y):
        if isinstance(y, NumpyTensor):
            return NumpyTensor(x.values.dot(y.values))
        elif isinstance(y, int):
            return NumpyTensor(x.values.dot(y))
        elif isinstance(y, float):
            return NumpyTensor(x.values.dot(y))
        elif isinstance(y, np.ndarray):
            return NumpyTensor(x.values.dot(y))
        else:
            print(type(y))
            raise Error
        
    def transpose(x):
        return NumpyTensor(x.values.T)
    
    def neg(x):
        return NumpyTensor(0 - x.values)

    def sum0(x):
        return NumpyTensor(x.values.sum(axis=0, keepdims=True))
    
    def sum1(x):
        return NumpyTensor(x.values.sum(axis=1, keepdims=True))
    
    def exp(x):
        return NumpyTensor(np.exp(x.values))
    
    def log(x):
        return NumpyTensor(np.log(x.values))

In [12]:
# x = NumpyTensor.load(np.array([1,2,3,4]).reshape(4,1) - 2)
# y = NumpyTensor.load(np.array([5,6,7,8]).reshape(4,1))
# z = x + y; print(z)
# z = (x * y); print(z)
# z = x.dot(y.transpose()); print(z)
# z = z.sum0(); print(z)